In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [2]:
data_train, meta = scipy.io.arff.loadarff('scene-train.arff')
df_train = pd.DataFrame(data_train)
data_test, meta = scipy.io.arff.loadarff('scene-test.arff')
df_test = pd.DataFrame(data_test)
df_test.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Att291,Att292,Att293,Att294,Beach,Sunset,FallFoliage,Field,Mountain,Urban
0,0.983225,0.981488,0.978542,0.975908,0.972962,0.968367,0.965900,0.938807,0.888701,0.862884,...,0.002494,0.012225,0.045861,0.094455,b'1',b'0',b'0',b'0',b'1',b'0'
1,0.018160,0.074481,0.369500,0.454558,0.423218,0.379960,0.661803,0.653965,0.666010,0.671838,...,0.005567,0.014649,0.025994,0.024701,b'1',b'0',b'0',b'0',b'0',b'0'
2,0.762800,0.791230,0.799371,0.795679,0.779932,0.756109,0.719042,0.877675,0.907015,0.919592,...,0.040818,0.051593,0.057795,0.057418,b'1',b'0',b'0',b'0',b'0',b'0'
3,0.758518,0.738699,0.709982,0.686717,0.694709,0.687867,0.461537,0.852653,0.831340,0.798184,...,0.599021,0.068253,0.061698,0.053979,b'1',b'0',b'0',b'0',b'1',b'0'
4,0.948445,0.969889,0.965691,0.969223,0.969579,0.966511,0.949118,0.823672,0.853659,0.857751,...,0.001599,0.000875,0.000813,0.003186,b'1',b'0',b'0',b'0',b'0',b'0'


In [3]:
X_train = df_train.iloc[:,:-6].values
y_train = df_train.iloc[:,-6:].values.astype(int)
X_test = df_test.iloc[:,:-6].values
y_test = df_test.iloc[:,-6:].values.astype(int)

In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1211, 294)
Train_Y:  (1211, 6)
Test_X:  (1196, 294)
Test_Y:  (1196, 6)


In [5]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Chain Model type 1


In [6]:
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session

Using TensorFlow backend.


In [7]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='first'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='last'))
    model.add(Dropout(0.4))    
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='output'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

def create_copynode(X_feed,y_now,w_name):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='copyfirst'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',trainable=False,name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',trainable=False,name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='copylast'))
    model.add(Dropout(0.4)) 
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='copyoutput'))
    model.load_weights(w_name,by_name=True)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [9]:
from time import time

In [11]:
#Model chain
chain = []
first = 1
#Training
X_feed = X_train.copy()
t1 = time()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))
    if first == 1:
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=20,batch_size=50)
        '''
        json_fl = node.to_json()
        with open('node1.json','w') as f:
            f.write(json_fl)
        '''
        node.save_weights('weights.h5')
        first = 0
    else:
        node = create_copynode(X_feed,y_now,'weights.h5')
        node.fit(X_feed,y_now,epochs=5,batch_size=50)
        
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)
t2 = time()

print("Time taken: ",(t2-t1))

Training chain node  0
Shapes:
 X = (1211, 294) 
 Y = (1211, 1)
Epoch 1/20
1211/1211 [==============================] - 2s 2ms/step - loss: 0.4834 - acc: 0.7861
Epoch 2/20
1211/1211 [==============================] - 0s 173us/step - loss: 0.4150 - acc: 0.8167
Epoch 3/20
1211/1211 [==============================] - 0s 166us/step - loss: 0.3417 - acc: 0.8332
Epoch 4/20
1211/1211 [==============================] - 0s 170us/step - loss: 0.3182 - acc: 0.8398
Epoch 5/20
1211/1211 [==============================] - 0s 186us/step - loss: 0.3040 - acc: 0.8679
Epoch 6/20
1211/1211 [==============================] - 0s 164us/step - loss: 0.2930 - acc: 0.8728
Epoch 7/20
1211/1211 [==============================] - 0s 170us/step - loss: 0.3053 - acc: 0.8712
Epoch 8/20
1211/1211 [==============================] - 0s 178us/step - loss: 0.2667 - acc: 0.8794
Epoch 9/20
1211/1211 [==============================] - 0s 215us/step - loss: 0.2466 - acc: 0.8910
Epoch 10/20
1211/1211 [========================

In [27]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    output = output.round().astype(int)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (1196, 294)
Getting op from chain node  1
Shapes:
 X = (1196, 295)
Getting op from chain node  2
Shapes:
 X = (1196, 296)
Getting op from chain node  3
Shapes:
 X = (1196, 297)
Getting op from chain node  4
Shapes:
 X = (1196, 298)
Getting op from chain node  5
Shapes:
 X = (1196, 299)


In [28]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [29]:
from sklearn.metrics import accuracy_score,hamming_loss, log_loss, f1_score

In [30]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.641304347826
Hamming accuracy =  0.688266443701
Hamming loss =  0.10214604236343366
Log loss =  10.6707192484
F1 score =  0.711331394693
